<a href="https://colab.research.google.com/github/AlexandreMenezesSa/pyspark/blob/main/spark_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark sql**

In [ ]:
# instalar pyspark
!pip install pyspark

In [ ]:
#faz importação dos pacotes e inicia a sessão
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.sql.types import *
from pyspark.sql import functions as Func
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[*]").appName("PySpark").getOrCreate()

In [ ]:
#mostrar bancos de dados e tabelas
spark.sql("show databases").show()


In [ ]:
#criar banco de dados
spark.sql("create database desp")
spark.sql("use desp").show()

In [ ]:
#criar tabela gerenciada
arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
despachantes = spark.read.csv("despachantes.csv", header=False, schema=arqschema)
despachantes.write.saveAsTable("Despachantes")

In [ ]:
#mostrar que a tabela existe
spark.sql("select * from despachantes").show()

In [ ]:
#mostra tabela
spark.sql("show tables").show()

In [ ]:

#mudar banco de dados
spark.sql("use default").show()